In [ ]:
import csv
import pandas as pd

In [ ]:
truth = pd.read_csv('True.csv')
fake = pd.read_csv('Fake.csv')
truth.head() #Returns n rows, default first 5 rows

In [ ]:
fake.head()    

In [ ]:
truth['label'] = 1
fake['label'] = 0
truth.head()

In [ ]:
news = pd.concat([truth, fake], axis = 0)        #axis zero is for concatenating in the rows wise
                                                 #Mixing both the fake and true news

news.head()

In [ ]:
news.isnull().sum()

In [ ]:
news = news.drop(['title', 'subject','date'], axis = 1)     #title, subject and date column is not required for the project
#drops the columns as mentioned (axis = 1 is for column wise)
news.head()

In [ ]:
news = news.sample(frac = 1)    #reshuffling
#frac here is responsible for the percentage of data you want to reshuffle, frac = 1 is 100% data
news.head() 

In [ ]:
news.reset_index(inplace =True)      #reset_index function resets the index
                                     #inplace = True,  Updates the news DataFrame in place instead of returning a new one.
                                     #for False, it would return a copy of the news DataFrame modified
news.head()

In [ ]:
news.drop(['index'], axis = 1, inplace =True)      #The old data index was invalid now so we got rid of it
news.head()

In [ ]:
#Converting into numerical data would be better since ML responds and processes numerical data better
#Convert them into numerical vectors, through this we can assign some weight to few important texts
#Making it code readable by the machine

In [ ]:
import re
#The re module in Python is used for working with regular expressions. It helps you find, replace, and manipulate text patterns efficiently

In [ ]:
def wordrop(text):
    #Making text clean, models don't work on raw test
    # Convert into lowercase
    text = text.lower()
    
    """We use patterns with re module. The pattern for url which has been used will be available in its documentation,
    memorizing is not necessary"""
    
    # Removes URLs
    text = re.sub(r"https?://\S+|www\.\S+", "", text)
    
    # Removes HTML tags
    text = re.sub(r'<.*?>', '', text)  

     # Removes digits
    text = re.sub(r'\d+', '', text) 

    # Removes punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # Removes newline characters
    text = re.sub(r'\n+', ' ', text)

    return text

    
   

In [ ]:
news['text'] = news['text'].apply(wordrop)           #Updating the text in the DataFrame

In [ ]:
news['text']

In [ ]:
x = news['text']
y = news['label']

In [ ]:
x

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split     #function for splitting datasets into training and testing sets!

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3)    #using 30% of data for testing, and 70% for training

In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer 
#used for converting text into numerical values using the TF-IDF (Term Frequency-Inverse Document Frequency) technique.


In [ ]:
vectorization = TfidfVectorizer()

In [ ]:
xv_train = vectorization.fit_transform(x_train)

In [ ]:
xv_test = vectorization.transform(x_test)

In [ ]:
xv_train

In [ ]:
xv_test


In [ ]:
#Creating a ML model, algorithm over this data
#Logistic Regression,  a statistical model commonly used for classification tasks in machine learning
from sklearn.linear_model import LogisticRegression
#predicts categories, uses probability based approach

In [ ]:
LR = LogisticRegression(solver = "saga")  #Creating instance, solver saga for handingling CSR format data
LR.fit(xv_train ,y_train)

In [ ]:
pred_lr = LR.predict(xv_test)

In [ ]:
LR.score(xv_test, y_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, pred_lr))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
# It builds a tree-like structure to decide which category a given input belongs to.
DTC = DecisionTreeClassifier()
DTC.fit(xv_train, y_train)

In [ ]:
pred_dtc = DTC.predict(xv_test)

In [ ]:
DTC.score(xv_test, y_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, pred_dtc))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# combines multiple decision trees to improve accuracy and reduce overfitting. It works by creating a "forest" of decision trees and averaging their predictions.


In [ ]:
rfc = RandomForestClassifier()
rfc.fit(xv_train, y_train)
predict_rfc = rfc.predict(xv_test)

In [ ]:
rfc.score(xv_test, y_test)

In [ ]:
print(classification_report(y_test,predict_rfc))

In [ ]:
#another algorithm
from sklearn.ensemble import GradientBoostingClassifier
#ensemble learning technique that builds multiple weak models (typically decision trees) and boosts their performance by focusing on hard-to-classify examples.

gbc = GradientBoostingClassifier()
gbc.fit(xv_train, y_train)

In [ ]:
preed_gbc = gbc_predict(xv_test)
gbc.score(xv_test, y_test)

In [ ]:
print(classification_report(y_test,preed_gbc))

In [ ]:
def output_label(n):
    if n == 0:
        return "It is a Fake News"
    elif n == 1:
        return "It is a Genuine News"

In [ ]:
def manual_testing(news):
    testing_news = {"text": [news]}   
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordrop)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_lr = LR.predict(new_xv_test)
    pred_gbc = gbc.predict(new_xv_test)
    pred_rfc = rfc.predict(new_xv_test)
    return "\n\nLR Prediction: {}  \nGBC Prediction:  {}  \nRFC  Prediction:  {}".format(output_label(pred_lr[0]),  output_label(pred_gbc[0]), output_label(pred_rfc[0]))

In [ ]:
news_article = str(input("Enter news article"))


In [ ]:
manual_testing(news_article)